# **Exploratory Data Analysis**

## **1. Introduction**

### **Notebook Overview**

---

### **EDA Objectives**

---

### **Key Questions to Explore**

---

### **Importing Packages and Configuring Directory Pathing**

In [ ]:
# REMOVE THIS BEFORE SUBMISSION
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from joblib import Parallel, delayed

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from jobrec import config
from jobrec import visualizer

In [ ]:
# for other theme, please run: mpl.pyplot.style.available
PLOT_PALETTE = 'tableau-colorblind10'
# for other color map, please run: mpl.pyplot.colormaps()
WORDCLOUD_COLOR_MAP = 'tab10_r'

---

## **2. Data Overview**

---

## **3. Job Listings EDA**

---

## **4. Resume EDA**

In [ ]:
# Use all cores
results = Parallel(n_jobs=-1)(
    delayed(generate_wordcloud_from_df)(df_categories[i], categories[i])
    for i in range(len(categories))
)

In [ ]:
resume_df['Category'].value_counts().sort_index().plot(kind='bar', figsize=(12, 6))
plt.show()

In [ ]:
plt.figure(figsize=(32, 28))

for i, (category, wc) in enumerate(results):
    plt.subplot(6, 4, i + 1).set_title(category)
    plt.imshow(wc)
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(32, 64))

for i, category in enumerate(categories):
    wf = wordfreq(df_categories[i])

    fig.add_subplot(12, 2, i + 1).set_title(category)
    plt.bar(wf['Word'], wf['Frequency'])
    plt.ylim(0, 1500)

plt.show()

---

## **5. Comparison**

---

## **6. Conclusion and Insights**